# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
# Loading the the Weather Csv and export it into a DateFrame
file_to_load = "../WeatherPy/WeatherPy_Outputs/My_Cities_Data.csv"
vacations_cities = pd.read_csv(file_to_load)
vacations_cities.head()

,City,Latitude,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aasiaat,68.7098,-52.8699,28.36,83,100,2.44,GL,1619742918
1,Abéché,13.8292,20.8324,80.26,9,20,6.73,TD,1619742919
2,Abhā,18.2164,42.5053,60.80,77,40,6.53,SA,1619742919
3,Abnūb,27.2696,31.1511,73.40,33,0,4.61,EG,1619742919
4,Abu Kamal,34.4506,40.9171,77.31,32,0,10.00,SY,1619742681


In [23]:
# Getting the Max Humidity
max_humidity = vacations_cities["Humidity"].max()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

In [25]:
# Storing locations
locations = vacations_cities[["Latitude","Lng"]]

# Setting Humidity as Weight
humidity = vacations_cities["Humidity"].astype(float)

In [26]:
# Plot Heatmap
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, gradient =["rgba(0, 255, 255, 0)","rgba(0, 255, 255, 1)",
                                                               "rgba(0, 191, 255, 1)","rgba(0, 127, 255, 1)",
                                                               "rgba(0, 63, 255, 1)","rgba(0, 0, 255, 1)",
                                                               "rgba(0, 0, 223, 1)","rgba(0, 0, 191, 1)",
                                                               "rgba(0, 0, 159, 1)","rgba(0, 0, 127, 1)",
                                                               "rgba(63, 0, 91, 1)","rgba(127, 0, 63, 1)",
                                                               "rgba(191, 0, 31, 1)","rgba(255, 0, 0, 1)",], 
                                 max_intensity=max_humidity,point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Creating a new DataFrame fitting the weather criteria
hotel_df=  vacations_cities.loc[(vacations_cities["Max Temperature"]<80) & 
                                               (vacations_cities["Max Temperature"]>70) & 
                                               (vacations_cities["Wind Speed"]<10) & 
                                               (vacations_cities["Cloudiness"]==0), :]

In [28]:
# Created DateFrame
hotel_df

,City,Latitude,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
3,Abnūb,27.2696,31.1511,73.40,33,0,4.61,EG,1619742919
83,Broken Hill,-31.9500,141.4333,71.60,28,0,9.22,AU,1619742952
104,Cayenne,4.9333,-52.3333,77.00,94,0,3.56,GF,1619742959
184,Guangyuan,32.4420,105.8230,70.61,65,0,2.30,CN,1619742987
216,Inhambane,-23.8650,35.3833,72.55,80,0,6.64,MZ,1619742764
272,Korla,41.7597,86.1469,71.46,16,0,3.96,CN,1619742787
281,La Libertad,-2.2333,-80.9000,75.20,83,0,9.22,EC,1619743021
315,Manicoré,-5.8092,-61.3003,75.20,100,0,1.14,BR,1619742803
414,Puqi,29.7167,113.8833,76.26,57,0,9.31,CN,1619742846
423,Riyadh,24.6877,46.7219,73.40,100,0,3.53,SA,1619742601


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# List of hotels within 5000 meters of the different locations
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels_list = []
lat_list = []
lng_list = []
loc_list = []
total_cities = hotel_df["City"].count()
z = 0
c = 0

for x in hotel_df["Latitude"]:
    lat_list.append(x)
for y in hotel_df["Lng"]:
    lng_list.append(y)

while c < total_cities:
    loc_list.append(f"{lat_list[c]},{lng_list[c]}")
    c = c + 1

while z < total_cities:
    target_coordinates = loc_list[z]
    target_radius = 5000
    target_type = "hotel"
    target_search = "hotel"
    
    params = {"location": target_coordinates,
              "keyword": target_search,
             "radius": target_radius,
             "type": target_type,
             "key": g_key}
    z = z + 1
    response = requests.get(base_url,params = params)
    hotels_data = response.json()
    try:
        print(hotels_data["results"][0]["name"])
        hotels_list.append(hotels_data["results"][0]["name"])
    except:
        print(f"Sorry, no hotels located within {target_radius} meters of your coordinates")
        hotels_list.append("NaN")
        pass


Sorry, no hotels located within 5000 meters of your coordinates
ibis Styles Broken Hill
Mercure Cayenne Royal Amazonia
Wanda Realm Guangyuan
Sentidos Beach Retreat
The Garden Hotel Korla
Playa Canela Hotel Boutique
Hotel Durval
Shanshui Trends Hotel
Four Seasons Hotel Riyadh At Kingdom Center
Amazone Lodges Oyack
El Salam Hotel


In [30]:
# Adding the list of hotels into my DataFrame
hotel_df["Hotel"] = hotels_list

hotel_df

<ipython-input-30-c568e7c87d73>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel"] = hotels_list


,City,Latitude,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
3,Abnūb,27.2696,31.1511,73.40,33,0,4.61,EG,1619742919,NaN
83,Broken Hill,-31.9500,141.4333,71.60,28,0,9.22,AU,1619742952,ibis Styles Broken Hill
104,Cayenne,4.9333,-52.3333,77.00,94,0,3.56,GF,1619742959,Mercure Cayenne Royal Amazonia
184,Guangyuan,32.4420,105.8230,70.61,65,0,2.30,CN,1619742987,Wanda Realm Guangyuan
216,Inhambane,-23.8650,35.3833,72.55,80,0,6.64,MZ,1619742764,Sentidos Beach Retreat
272,Korla,41.7597,86.1469,71.46,16,0,3.96,CN,1619742787,The Garden Hotel Korla
281,La Libertad,-2.2333,-80.9000,75.20,83,0,9.22,EC,1619743021,Playa Canela Hotel Boutique
315,Manicoré,-5.8092,-61.3003,75.20,100,0,1.14,BR,1619742803,Hotel Durval
414,Puqi,29.7167,113.8833,76.26,57,0,9.31,CN,1619742846,Shanshui Trends Hotel
423,Riyadh,24.6877,46.7219,73.40,100,0,3.53,SA,1619742601,Four Seasons Hotel Riyadh At Kingdom Center


In [31]:
# Handling in case there is no hotel found in the specific radius
hotel_df =  hotel_df.loc[hotel_df["Hotel"] != "NaN",:]
hotel_df

,City,Latitude,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
83,Broken Hill,-31.9500,141.4333,71.60,28,0,9.22,AU,1619742952,ibis Styles Broken Hill
104,Cayenne,4.9333,-52.3333,77.00,94,0,3.56,GF,1619742959,Mercure Cayenne Royal Amazonia
184,Guangyuan,32.4420,105.8230,70.61,65,0,2.30,CN,1619742987,Wanda Realm Guangyuan
216,Inhambane,-23.8650,35.3833,72.55,80,0,6.64,MZ,1619742764,Sentidos Beach Retreat
272,Korla,41.7597,86.1469,71.46,16,0,3.96,CN,1619742787,The Garden Hotel Korla
281,La Libertad,-2.2333,-80.9000,75.20,83,0,9.22,EC,1619743021,Playa Canela Hotel Boutique
315,Manicoré,-5.8092,-61.3003,75.20,100,0,1.14,BR,1619742803,Hotel Durval
414,Puqi,29.7167,113.8833,76.26,57,0,9.31,CN,1619742846,Shanshui Trends Hotel
423,Riyadh,24.6877,46.7219,73.40,100,0,3.53,SA,1619742601,Four Seasons Hotel Riyadh At Kingdom Center
425,Roura,4.7263,-52.3274,77.00,94,0,0.67,GF,1619742850,Amazone Lodges Oyack


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Lng"]]
humidity = hotel_df["Humidity"].astype(float)

In [33]:
# Add marker layer ontop of heat map
# Plot Heatmap
figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False,max_intensity=10,point_radius=1)


# Add layer
fig.add_layer(heat_layer)
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…